In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
gloabal_prot = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_processed_data/FFPE_discovery_globalprotein_imputed.tsv",delimiter='\t')
gloabal_prot.columns.to_list()


"H2BC20P" in gloabal_prot["HGNC symbol"].to_list() #here its true...

True

In [3]:
clinical = pd.read_excel("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_processed_data/PTRC-HGSOC_List_clincal_data.xlsx")
len(clinical["Sample ID"].unique()) # 174... samles, 159 patients... 168 genmics cols.... 
clinical.head()

,File Name,Patient ID,Sample Source,Sample ID,Image Name,Image ID,Patient Age,Patient Ethnicity,Patient Race,Neo-adjuvant,Other cancer diagnosis?,Tumor response,Tumor type,Tumor location,Tumor Location Group,Tumor Grade,Tumor Stage,Tumor Substage,Sample type
0,1009402_171069_ImageActual,2,FHCRC,205927,1009402,171069,73,Not Hispanic or Latino,White,no,NaN,refractory,Metastatic,Omentum,OM,3,III,IIIC,FFPE
1,1009403_171101_ImageActual,2,FHCRC,205927,1009403,171101,73,Not Hispanic or Latino,White,no,NaN,refractory,Metastatic,Omentum,OM,3,III,IIIC,FFPE
2,1009404_171121_ImageActual,4,FHCRC,206053,1009404,171121,60,NaN,NaN,no,NaN,sensitive,Primary,Ovary - Left,OV,2,III,IIIC,FFPE
3,1009405_416622_ImageActual,4,FHCRC,206053,1009405,416622,60,NaN,NaN,no,NaN,sensitive,Primary,Ovary - Left,OV,2,III,IIIC,FFPE
4,1009406_171004_ImageActual,7,FHCRC,206382,1009406,171004,52,Not Hispanic or Latino,NaN,no,NaN,refractory,Primary,Ovary - Left,OV,3,III,IIIC,FFPE


In [4]:
gloabal_prot_t = gloabal_prot.T

# drop middle rows .... make header the prot name... 
gloabal_prot_t.columns = gloabal_prot_t.iloc[1]


print(gloabal_prot_t.columns)
print("H2AC20" in gloabal_prot_t.columns.to_list()) #here its true...

gloabal_prot_t = gloabal_prot_t.iloc[9:]

row_to_subtract = gloabal_prot_t.iloc[0]
# Subtract this row from all other rows
df_subtracted = gloabal_prot_t.subtract(row_to_subtract, axis='columns')


df_subtracted = df_subtracted.iloc[1:]
# norm standaard scalar
scaler = StandardScaler()
# Fit and transform the DataFrame
df_subtracted = pd.DataFrame(scaler.fit_transform(df_subtracted), columns=df_subtracted.columns, index=df_subtracted.index)

df_subtracted["Sample ID1"] = df_subtracted.index

# print(df_subtracted["Sample ID1"].value_counts()) # 168 patients.... but some not in other row?.,... maybe duplicates for the same patient? Or done with different divices to mearue....

# Join with clinical now....
def custom_join(df1, df2, column1, column2):
    # Create a new DataFrame to store the result
    df1[column1] = df1[column1].astype(str)
    df2[column2] = df2[column2].astype(str)

    # Create a new DataFrame to store the result
    result_df = pd.DataFrame()

    for index, row in df1.iterrows():
        matching_rows = df2[df2[column2].apply(lambda x: x in row[column1])]
        for _, match in matching_rows.iterrows():
            result_row = pd.concat([row, match])
            result_df = result_df.append(result_row, ignore_index=True)

    return result_df


# special moerging on substrings... 
print(len(clinical)) #348

result = custom_join(df_subtracted, clinical, 'Sample ID1', 'Sample ID')

Index(['A1BG', 'A2M', 'AAAS', 'AACS', 'AAGAB', 'AAK1', 'AAMDC', 'AAMP', 'AAR2',
       'AARS1',
       ...
       'ZSCAN31', 'ZSWIM8', 'ZSWIM9', 'ZW10', 'ZWILCH', 'ZWINT', 'ZYG11B',
       'ZYX', 'ZZEF1', 'ZZZ3'],
      dtype='object', name='HGNC symbol', length=8800)
True
348


In [5]:
result['label'] = result['Tumor response'].map({'sensitive': 1, 'refractory': 0, 'Sensitive': 1, 'Refractory': 0})

In [6]:
headers = ["File Name","Patient ID","label","Tumor type"]

all_prots = result.iloc[:, 0:8800].columns 
prots = all_prots.to_list()

for prot in all_prots:
    headers.append(prot)

final_df = result[headers]
final_df = result.rename(columns={'File Name': 'slide_id', 'Patient ID': 'case_id'})
final_df.head()

,A1BG,A2M,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,AAR2,AARS1,...,Other cancer diagnosis?,Tumor response,Tumor type,Tumor location,Tumor Location Group,Tumor Grade,Tumor Stage,Tumor Substage,Sample type,label
0,2.581553,2.142062,0.403025,0.940590,0.089046,-0.463530,-0.787507,0.275762,0.291986,-0.894981,...,NaN,refractory,Metastatic,omental cake,OM,3.0,IV,NaN,FFPE,0
1,2.581553,2.142062,0.403025,0.940590,0.089046,-0.463530,-0.787507,0.275762,0.291986,-0.894981,...,NaN,refractory,Metastatic,omental cake,OM,3.0,IV,NaN,FFPE,0
2,-0.636902,-1.654378,-0.485982,0.938202,0.399242,0.645449,1.666918,1.905756,1.145809,0.373713,...,NaN,sensitive,Primary,Ovary - Left,OV,3.0,III,IIIA,FFPE,1
3,-0.636902,-1.654378,-0.485982,0.938202,0.399242,0.645449,1.666918,1.905756,1.145809,0.373713,...,NaN,sensitive,Primary,Ovary - Left,OV,3.0,III,IIIA,FFPE,1
4,-0.239903,-0.024764,-0.469240,-1.443448,-1.338855,0.157796,-0.602167,-1.735674,-1.291855,-1.584350,...,NaN,sensitive,Metastatic,peritoneal biopsy,PT,3.0,III,NaN,FFPE,1


In [7]:
len(final_df) #... but only 348 slides.......?

df_unique = final_df.drop_duplicates(subset="slide_id")

df_unique["slide_id"].value_counts()

# take just one from each....
df_unique.to_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/HGSOC_multimodal_all_prots_oz_merge.csv",index=None)

In [8]:
# len(df_unique.columns.to_list())

are_duplicates = df_unique.columns.duplicated().any()
are_duplicates

False

### Test the classical model that there is signal here

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, roc_auc_score



prots = ['ABCB8', 'ACADSB', 'ACOT7', 'AIFM1', 'ARNT', 'ATM',
       'ATP6V1D', 'BCL2L1', 'BIRC2', 'CA2', 'CAMK2D', 'CARMIL1', 'CASP7',
       'CCDC167', 'CDKN1A', 'CDKN1B', 'CHUK', 'COX6A1', 'COX6C', 'COX7A2',
       'COX7C', 'CPOX', 'EGFR', 'FDX1', 'GLUD1', 'GOLPH3L', 'GPX1', 'GPX4',
       'HACL1', 'HADH', 'IDH1', 'IDI1', 'IL4I1', 'KEAP1', 'LDHA', 'LGALS1',
       'LTA4H', 'MTDH', 'MYD88', 'NBN', 'NCAPH2', 'NDUFB3', 'NFKB1', 'NFKB2',
       'PDK1', 'PECR', 'PLIN2', 'PTGES2', 'RAB25', 'RELA', 'RHOA', 'RIPK1',
       'RPS6KB2', 'S100A10', 'SENP1', 'STAT3', 'TALDO1', 'TFRC', 'TGM2',
       'TOP3B', 'TPMT', 'VCAM1', 'XIAP', 'YWHAG']


# Define the column you want to split by
group_column = 'Sample ID'

# Create a list of unique groups
unique_groups = df_unique[group_column].unique()

# Split the unique groups into training and test sets
train_groups, test_groups = train_test_split(unique_groups, test_size=0.2, random_state=42)

# Create training and test DataFrames
train_df = df_unique[df_unique[group_column].isin(train_groups)]
test_df = df_unique[df_unique[group_column].isin(test_groups)]


# train_df, test_df  = train_test_split(result, test_size=0.2, random_state=42, stratify=result["Sample ID"])


X_train = train_df[prots]
X_test =  test_df[prots]
y_train = train_df['label']
y_test = test_df['label']


print(y_train)

# Initialize and train the Random Forest classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
rf_pred = rf_clf.predict(X_test)
rf_pred_proba = rf_clf.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Evaluate the Random Forest model
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_roc_auc = roc_auc_score(y_test, rf_pred_proba)
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest ROC AUC: {rf_roc_auc}")

0      0
1      0
2      1
3      1
4      1
      ..
361    0
364    1
365    1
366    1
367    1
Name: label, Length: 278, dtype: int64
Random Forest Accuracy: 0.7428571428571429
Random Forest ROC AUC: 0.8766666666666666


In [10]:
# append the other df's?
df_unique['label'].value_counts()

1    202
0    146
Name: label, dtype: int64

In [11]:

clinical["Tumor response"].value_counts()


sensitive     156
refractory    100
Refractory     46
Sensitive      46
Name: Tumor response, dtype: int64